In [3]:
import joblib
import warnings

warnings.filterwarnings('ignore')

path='./best_model/Linear_Cassava_Model.pkl'
model = joblib.load(path)

In [4]:
import pandas as pd

data = {
    'month': [8],
    'plant': ["A003"],
    'vendor': ["500005005"],
    'fines': [12.5],
    'bulk': [0.555]
}

df = pd.DataFrame(data)
df

,month,plant,vendor,fines,bulk
0,8,A003,500005005,12.5,0.555


In [5]:
sand_predict_value = model.predict(df)
sand_value = float(sand_predict_value[0])
total_sand_value = (sand_value * df['fines'].iloc[0]) / 100
total_sand_value

3.676699999999997

In [6]:
import pandas as pd
import psycopg2
from psycopg2 import extras
from config import settings
import logging

# Initialize your logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_connection():
    try:
        conn = psycopg2.connect(
            host=settings.predict_db_host,
            port=settings.predict_db_port,  # Corrected from predict_connect_db_port
            dbname=settings.predict_db_name,
            user=settings.predict_db_username,
            password=settings.predict_db_password
        )
        return conn
    except psycopg2.Error as e:
        logger.error(f"Error connecting to PostgreSQL database: {e}")
        return None

def insert_batch_info(data):
    query = """
    INSERT INTO prediction.batch_info (inslot, material, batch, plant, operationno)
    VALUES %s
    ON CONFLICT (inslot, material, batch, plant, operationno) DO NOTHING;
    """
    conn = create_connection()
    if conn:
        try:
            with conn.cursor() as cursor:
                extras.execute_values(cursor, query, data)
                conn.commit()
                logger.info("Data inserted successfully into batch_info.")
        except Exception as e:
            logger.error(f"Error inserting data into batch_info: {e}")
        finally:
            conn.close()

def insert_result(data):
    query = """
    INSERT INTO prediction.result (inslot, material, batch, plant, operationno, month, fines, bulk, sand_predict_value, total_sand_value)
    VALUES %s
    ON CONFLICT (inslot, material, batch, plant, operationno) DO NOTHING;
    """
    conn = create_connection()
    if conn:
        try:
            with conn.cursor() as cursor:
                extras.execute_values(cursor, query, data)
                conn.commit()
                logger.info("Data inserted successfully into result.")
        except Exception as e:
            logger.error(f"Error inserting data into result: {e}")
        finally:
            conn.close()

def insert_mic_result(data):
    query = """
    INSERT INTO prediction.mic_result (inslot, material, batch, plant, operationno, phys0001, chem0010, chem0013)
    VALUES %s
    ON CONFLICT (inslot, material, batch, plant, operationno) DO NOTHING;
    """
    conn = create_connection()
    if conn:
        try:
            with conn.cursor() as cursor:
                extras.execute_values(cursor, query, data)
                conn.commit()
                logger.info("Data inserted successfully into mic_result.")
        except Exception as e:
            logger.error(f"Error inserting data into mic_result: {e}")
        finally:
            conn.close()

def select_data_by_vendor(vendor):
    query = """
    SELECT * FROM prediction.masters WHERE vendor = %s;
    """
    conn = create_connection()
    if conn:
        try:
            with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cursor:
                cursor.execute(query, (vendor,))
                result = cursor.fetchall()
                df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
                return df
        except Exception as e:
            logger.error(f"Error selecting data from masters: {e}")
            return None
        finally:
            conn.close()

# Example data for testing
batch_info_data = [
    ('slot1', 'material1', 'batch1', 'plant1', 'operation1'),
    ('slot2', 'material2', 'batch2', 'plant2', 'operation2')
]

result_data = [
    ('slot1', 'material1', 'batch1', 'plant1', 'operation1', 5, 10.5, 0.3, 15.0, 20.0),
    ('slot2', 'material2', 'batch2', 'plant2', 'operation2', 6, 12.5, 0.4, 16.0, 22.0)
]

mic_result_data = [
    ('slot1', 'material1', 'batch1', 'plant1', 'operation1', 0.1, 0.2, 0.3),
    ('slot2', 'material2', 'batch2', 'plant2', 'operation2', 0.4, 0.5, 0.6)
]

# Insert data
insert_batch_info(batch_info_data)
insert_result(result_data)
insert_mic_result(mic_result_data)

# Select data
df = select_data_by_vendor('500005005')
if df is not None:
    print(df)


ERROR:__main__:Error connecting to PostgreSQL database: could not translate host name "prediction-db" to address: Temporary failure in name resolution

ERROR:__main__:Error connecting to PostgreSQL database: could not translate host name "prediction-db" to address: Temporary failure in name resolution

ERROR:__main__:Error connecting to PostgreSQL database: could not translate host name "prediction-db" to address: Temporary failure in name resolution

ERROR:__main__:Error connecting to PostgreSQL database: could not translate host name "prediction-db" to address: Temporary failure in name resolution

